# Demonstration for the upstream regulation network reconstruction

This notebook demonstrates how to use BRAvo to reconstruct a regulation network with specific options and use it as a model, here to infer new knownledge from partial knowledge on nodes evolution.

In [1]:
# Imports
import time
import csv
import logging
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

## Network reconstruction

### Global options

We define below the global options regarding the execution:

- Max depth of 10 levels
- Fast reconstruction (look only at standard names)
- Decompose complexes
- Extend search by considering synonyms and removing suffixes
- Include unsinged interactions
- Use Pathway Commons' SPARQL endpoint
- Consider all data sources except MiRTarBase and MSigDB

In [2]:
# Global options

config.MAX_DEPTH = 10
config.HAS_MAX_DEPTH = True
config.FAST = True
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.UNKNOWN = True
config.SPARQL_ENDPOINT = "http://rdf.pathwaycommons.org/sparql/"
config.VERBOSE = True

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
      'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
      'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
      'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
      'intact_complex', 'msigdb', 'reach', 'wikipathways']

config.DATA_SOURCES = set(ds) - set(['mirtarbase', 'msigdb'])

pybravo_stdout = [''] * 3

### Gene names input list

Here we read the list of input gene names from a file called `input-910.csv`. This file should contain one gene name per line.

In [3]:
# Read input genes list from a file
def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Genes input list
gene_list = read_input_genes('reg-demo/data/input-910.csv')
print('{} genes in input list'.format(len(gene_list)))

910 genes in input list


### Initial reconstruction

We now call the reconstruction on the previous list of gene names.
This iterative step is performed by querying the Pathway Commons endpoint and using the results to complete the graph and prepare the next query.
It may take some time, typically between 20 and 30 minutes.

In [4]:
# Add logging handler to file for later analysis
handler = logging.FileHandler('reg-demo/std.log', mode = 'w')
handler.setFormatter(logging.Formatter('%(message)s'))
logging.getLogger().addHandler(handler)

In [5]:
%%capture --no-display pybravo_stdout_0

start_time = time.time()

# Call reconstruction
reconstructed_network = regulation.upstream_regulation(gene_list)

elapsed_time = round((time.time() - start_time), 2)

2019-12-06 11:51:03,412 - INFO: to be explored after complex decomposition ['LUM', 'CPZ', 'DCN', 'ISLR', 'FNDC1', 'THBS2', 'COL16A1', 'COL1A1', 'COL3A1', 'COL5A1', 'SSC5D', 'MMP28', 'GEM', 'DACT3', 'MRC2', 'GPR124', 'LOXL1', 'ST6GAL2', 'COL1A2', 'PRICKLE1', 'LTBP2', 'COL6A3', 'CDR2L', 'PMP22', 'EMILIN1', 'COL14A1', 'CDH11', 'C1QTNF2', 'CCDC80', 'EFEMP1', 'GLT8D2', 'GGT5', 'ANTXR1', 'SNAI1', 'TSHZ3', 'PPAPDC1A', 'ALDH1A3', 'HEPH', 'FBN1', 'ECM1', 'DPYSL3', 'SRPX', 'PODN', 'GFPT2', 'FOXL1', 'COL10A1', 'GAS1', 'ITGA11', 'GLI2', 'HAND2', 'VCAN', 'GPX8', 'POSTN', 'CRISPLD2', 'PTGIR', 'CALHM2', 'FOXF1', 'MOXD1', 'MGP', 'ASPN', 'COL12A1', 'ZNF469', 'CHST3', 'NHS', 'LAMA2', 'AEBP1', 'PCDH7', 'HTRA3', 'HDGFRP3', 'MXRA5', 'HEYL', 'SVEP1', 'MAP1A', 'SPEG', 'GLI3', 'COL6A2', 'F2R', 'FAP', 'C13orf33', 'SEMA3C', 'ALDH1L2', 'LXN', 'PHLDB1', 'PRELP', 'ADAMTS2', 'PMEPA1', 'EPHA3', 'FLNA', 'RSPO3', 'TGFB3', 'TMEM132E', 'SGCA', 'C1QTNF7', 'COL8A2', 'ADAMTS12', 'MFAP4', 'LAMA1', 'PLAT', 'CTBP2', 'COL4A2',

2019-12-06 11:51:03,414 - INFO: new synonmys to be explored:['LDC', 'SLRR2D', 'CSCD', 'DSPG2', 'PG40', 'PGII', 'PGS2', 'SLRR1B', 'HsT17563', 'Meflin', 'AGS8', 'FNDC2', 'MEL4B3', 'bA243O10.1', 'dJ322A24.1', 'TSP2', '447AA', 'FP1572', 'EDSARTH1', 'EDSC', 'OI1', 'OI2', 'OI3', 'OI4', 'EDS4A', 'EDSVASC', 'EDSC', 'EDSCL1', 'S5D-SRCRB', 'EPILYSIN', 'MM28', 'MMP-25', 'MMP-28', 'MMP25', 'KIR', 'DAPPER3', 'RRR1', 'CD280', 'CLEC13E', 'ENDO180', 'UPARAP', 'ADGRA2', 'TEM5', 'LOL', 'LOXL', 'SIAT2', 'ST6GalII', 'EDSARTH2', 'EDSCV', 'OI4', 'EPM1B', 'RILP', 'LTBP3', 'DASS', 'GPHYSD3', 'LTBP-3', 'STHAG6', 'pp6425', 'BTHLM1', 'DYT27', 'UCMD1', 'HUMPPA', 'PXMP2', 'EMI', 'EMILIN', 'gp115', 'UND', 'CAD11', 'CDHOB', 'ESWS', 'OB', 'OSF-4', 'CTRP2', 'zacrp2', 'DRO1', 'SSG1', 'URB', 'okuribin', 'DHRD', 'DRAD', 'FBLN3', 'FBNL', 'FIBL-3', 'MLVT', 'MTLV', 'S1-5', 'GGL', 'GGT 5', 'GGT-REL', 'GGTLA1', 'ATR', 'GAPO', 'TEM8', 'SLUGH2', 'SNA', 'SNAH', 'SNAIL', 'SNAIL1', 'dJ710H13.1', 'TSH3', 'ZNF537', 'PLPP4', 'DPPL2',

2019-12-06 11:51:23,637 - INFO: 		removing suffix  complex for NFX1/SIN3/HDAC complex
2019-12-06 11:51:23,640 - INFO: 				 --> NFX1/SIN3/HDAC
2019-12-06 11:51:53,068 - INFO: 		removing suffix  complex for tungsten carbide/Cobalt complex
2019-12-06 11:51:53,071 - INFO: 				 --> tungsten carbide/Cobalt
2019-12-06 11:51:53,072 - INFO: 		removing suffix  protein for DCN protein
2019-12-06 11:51:53,073 - INFO: 				 --> DCN
2019-12-06 11:51:53,073 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 11:51:53,074 - INFO: 				 --> NOG
2019-12-06 11:51:53,075 - INFO: 		removing suffix  protein for DCN protein
2019-12-06 11:51:53,076 - INFO: 				 --> DCN
2019-12-06 11:51:53,076 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 11:51:53,077 - INFO: 				 --> NOG
2019-12-06 11:51:53,078 - INFO: 		removing suffix  mRNA for LUM mRNA
2019-12-06 11:51:53,078 - INFO: 				 --> LUM
2019-12-06 11:51:53,079 - INFO: 		removing suffix  protein for INS protein
2019-12-06 11:51:53,080 - INF

2019-12-06 11:51:54,142 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 11:51:54,145 - INFO: 				 --> COL1A1
2019-12-06 11:51:54,146 - INFO: 		removing suffix  protein for IL6 protein
2019-12-06 11:51:54,147 - INFO: 				 --> IL6
2019-12-06 11:51:54,148 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 11:51:54,149 - INFO: 				 --> COL1A1
2019-12-06 11:51:54,149 - INFO: 		removing suffix  protein for DEFA1 protein
2019-12-06 11:51:54,150 - INFO: 				 --> DEFA1
2019-12-06 11:51:54,151 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 11:51:54,151 - INFO: 				 --> COL1A1
2019-12-06 11:51:54,152 - INFO: 		removing suffix  protein for LEP protein
2019-12-06 11:51:54,153 - INFO: 				 --> LEP
2019-12-06 11:51:54,153 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 11:51:54,154 - INFO: 				 --> COL1A1
2019-12-06 11:51:54,154 - INFO: 		removing suffix  protein for ABL1 protein
2019-12-06 11:51:54,155 - INFO: 				 --> ABL1
2019-12-06 11:51:54,155 - INFO: 		

2019-12-06 11:51:56,231 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2019-12-06 11:51:56,233 - INFO: 				 --> PMP22
2019-12-06 11:51:56,236 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 11:51:56,237 - INFO: 				 --> NOG
2019-12-06 11:51:56,237 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2019-12-06 11:51:56,238 - INFO: 				 --> COL6A3
2019-12-06 11:51:56,241 - INFO: 		removing suffix  protein for INS protein
2019-12-06 11:51:56,241 - INFO: 				 --> INS
2019-12-06 11:51:56,242 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2019-12-06 11:51:56,243 - INFO: 				 --> PMP22
2019-12-06 11:51:56,244 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 11:51:56,244 - INFO: 				 --> TGFB1
2019-12-06 11:51:56,245 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2019-12-06 11:51:56,246 - INFO: 				 --> COL6A3
2019-12-06 11:51:56,247 - INFO: 		removing suffix  protein for pirinixic acid/PPARA protein complex
2019-12-06 11:51:56,248 - INFO: 				 --> pirinixic acid/PPAR

...

2019-12-06 12:07:34,109 - INFO: 		removing suffix  protein for TP53 protein
2019-12-06 12:07:34,110 - INFO: 				 --> TP53
2019-12-06 12:07:34,112 - INFO: 		removing suffix  mutant form for TP53 gene mutant form
2019-12-06 12:07:34,113 - INFO: 				 --> TP53 gene
2019-12-06 12:07:34,113 - INFO: 		removing suffix  gene for TP53 gene
2019-12-06 12:07:34,114 - INFO: 				 --> TP53
2019-12-06 12:07:34,114 - INFO: 		removing suffix  protein for TP53 protein
2019-12-06 12:07:34,115 - INFO: 				 --> TP53
2019-12-06 12:07:34,116 - INFO: 		removing suffix  protein for PRKN protein
2019-12-06 12:07:34,117 - INFO: 				 --> PRKN
2019-12-06 12:07:34,118 - INFO: 		removing suffix  protein for TP53 protein
2019-12-06 12:07:34,118 - INFO: 				 --> TP53
2019-12-06 12:07:34,120 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 12:07:34,121 - INFO: 				 --> NOG
2019-12-06 12:07:34,121 - INFO: 		removing suffix  mRNA for AHR mRNA
2019-12-06 12:07:34,122 - INFO: 				 --> AHR
2019-12-06 12:07:34,12

2019-12-06 12:07:35,269 - INFO: 				 --> KLF4
2019-12-06 12:07:35,837 - INFO: 		removing suffix  protein for APP protein alternative form
2019-12-06 12:07:35,840 - INFO: 				 --> APP alternative form
2019-12-06 12:07:35,842 - INFO: 		removing suffix  protein for FOXO3 protein
2019-12-06 12:07:35,846 - INFO: 				 --> FOXO3
2019-12-06 12:07:35,852 - INFO: 		removing suffix  protein for INS protein
2019-12-06 12:07:35,855 - INFO: 				 --> INS
2019-12-06 12:07:35,859 - INFO: 		removing suffix  protein for FOXO3 protein
2019-12-06 12:07:35,862 - INFO: 				 --> FOXO3
2019-12-06 12:07:35,867 - INFO: 		removing suffix  mutant form for PIK3CA gene mutant form
2019-12-06 12:07:35,869 - INFO: 				 --> PIK3CA gene
2019-12-06 12:07:35,872 - INFO: 		removing suffix  gene for PIK3CA gene
2019-12-06 12:07:35,873 - INFO: 				 --> PIK3CA
2019-12-06 12:07:35,875 - INFO: 		removing suffix  protein for FOXO3 protein
2019-12-06 12:07:35,876 - INFO: 				 --> FOXO3
2019-12-06 12:07:35,879 - INFO: 		removing s

2019-12-06 12:08:20,883 - INFO: 				 --> INS
2019-12-06 12:08:20,883 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 12:08:20,884 - INFO: 				 --> MLXIPL
2019-12-06 12:08:20,886 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 12:08:20,887 - INFO: 				 --> TGFB1
2019-12-06 12:08:20,888 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 12:08:20,889 - INFO: 				 --> MLXIPL
2019-12-06 12:08:27,038 - INFO: 		removing suffix  protein for INS protein
2019-12-06 12:08:27,041 - INFO: 				 --> INS
2019-12-06 12:08:27,044 - INFO: 		removing suffix  mRNA for ZNF225 mRNA
2019-12-06 12:08:27,046 - INFO: 				 --> ZNF225
2019-12-06 12:08:27,755 - INFO: 		removing suffix  protein for INS protein
2019-12-06 12:08:27,758 - INFO: 				 --> INS
2019-12-06 12:08:27,761 - INFO: 		removing suffix  mRNA for TP53BP1 mRNA
2019-12-06 12:08:27,764 - INFO: 				 --> TP53BP1
2019-12-06 12:08:30,211 - INFO: MYC/Max/TRRAP/TIP60/TIP49A/TIP49B/BAF53 decomposed into ['MYC', 'Max', 'TRRAP

2019-12-06 12:08:37,122 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 12:08:37,123 - INFO: 				 --> MYC
2019-12-06 12:08:37,124 - INFO: 		removing suffix  complex for Niclosamide/Polyethyleneimine complex
2019-12-06 12:08:37,124 - INFO: 				 --> Niclosamide/Polyethyleneimine
2019-12-06 12:08:37,125 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 12:08:37,126 - INFO: 				 --> MYC
2019-12-06 12:08:37,127 - INFO: 		removing suffix  protein for CTNNB1 protein
2019-12-06 12:08:37,128 - INFO: 				 --> CTNNB1
2019-12-06 12:08:37,129 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 12:08:37,130 - INFO: 				 --> MYC
2019-12-06 12:08:37,131 - INFO: 		removing suffix  protein for AR protein
2019-12-06 12:08:37,132 - INFO: 				 --> AR
2019-12-06 12:08:37,132 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 12:08:37,133 - INFO: 				 --> MYC
2019-12-06 12:08:37,134 - INFO: 		removing suffix  mutant form for KRAS gene mutant form
2019-12-06 12:08:37,135 - INFO: 				 --

2019-12-06 12:08:46,526 - INFO: new synonmys to be explored:['Pro-TRH', 'TRF']
2019-12-06 12:08:47,414 - INFO: Exploring done


In [6]:
print("--- Upstream regulation network reconstructed in {} seconds ---".format(elapsed_time))

--- Upstream regulation network reconstructed in 1064.01 seconds ---


In [7]:
%%capture --no-display pybravo_stdout_1

# Build digraph representation in memory
G = bravo_main.build_nx_digraph(reconstructed_network)

bravo_main.write_to_SIF(G, 'reg-demo/out.sif')
bravo_main.write_provenance(G, 'reg-demo/out-provenance.csv')

### Synonym-based unification

We now perform a synonym-unification, which consists in fusing nodes together if they are synonyms.
This process is useful because of the high heterogeneity of the data sources that compose Pathway Commons.
It typically takes a few seconds.

In [8]:
%%capture --no-display pybravo_stdout_2

start_time = time.time()
# Perform unification
G_prime = util.fast_reg_network_unification(G, util.index_syn)
elapsed_time = round((time.time() - start_time), 2)
print("--- Network simplification in %s seconds ---" % elapsed_time)

# Write to files
bravo_main.write_to_SIF(G_prime, 'reg-demo/out-unified.sif')
bravo_main.write_provenance(G_prime, 'reg-demo/out-unified-provenance.csv')

# Count nodes/edges
print('Nodes after simplification = ' + str(len(G_prime.nodes())))
print('Edges after simplification = ' + str(len(G_prime.edges())))

# Compute centrality
print(bravo_main.get_centrality_as_md(G_prime))

2019-12-06 12:08:47,605 - INFO: 		merging node ZFP423 into node ZNF423
2019-12-06 12:08:47,638 - INFO: 		merging node JAB1 into node COPS5
2019-12-06 12:08:47,676 - INFO: 		merging node AP1 into node JUN
2019-12-06 12:08:47,691 - INFO: 		merging node c-Jun into node JUN
2019-12-06 12:08:47,707 - INFO: 		merging node AP1 into node JUN
2019-12-06 12:08:47,734 - INFO: 		merging node ATF2 into node GDNF
2019-12-06 12:08:47,749 - INFO: 		merging node ATF2 into node GDNF
2019-12-06 12:08:47,772 - INFO: 		merging node MIM into node MTSS1
2019-12-06 12:08:47,790 - INFO: 		merging node ROR1 into node RORA
2019-12-06 12:08:47,806 - INFO: 		merging node ROR2 into node RORA
2019-12-06 12:08:47,885 - INFO: 		merging node ROR1 into node RORA
2019-12-06 12:08:47,919 - INFO: 		merging node CD3G into node TARP
2019-12-06 12:08:47,938 - INFO: 		merging node FIAT into node TXLNG
2019-12-06 12:08:47,956 - INFO: 		merging node AGT into node AGXT
2019-12-06 12:08:47,975 - INFO: 		merging node REDD1 into nod

2019-12-06 12:08:51,054 - INFO: 		merging node BK2 into node BDKRB2
2019-12-06 12:08:51,071 - INFO: 		merging node FOXO3A into node FOXO3
2019-12-06 12:08:51,086 - INFO: 		merging node FEZ1 into node LZTS1
2019-12-06 12:08:51,103 - INFO: 		merging node RASSF5 into node RASSF3
2019-12-06 12:08:51,188 - INFO: 		merging node DP into node SRPRA
2019-12-06 12:08:51,208 - INFO: 		merging node FHOD3 into node FMNL3
2019-12-06 12:08:51,229 - INFO: 		merging node maspin into node SERPINB5
2019-12-06 12:08:51,250 - INFO: 		merging node JUNB into node RNASEH2A
2019-12-06 12:08:51,271 - INFO: 		merging node LKB1 into node STK11
2019-12-06 12:08:51,313 - INFO: 		merging node TCF3 into node TCF7L1
2019-12-06 12:08:51,334 - INFO: 		merging node RANTES into node CCL5
2019-12-06 12:08:51,350 - INFO: 		merging node PMP22 into node PXMP2
2019-12-06 12:08:51,393 - INFO: 		merging node CSPG2 into node VCAN
2019-12-06 12:08:51,410 - INFO: 		merging node TCF3 into node TCF7L1
2019-12-06 12:08:51,430 - INFO: 

### Saving output

Here we simply save all output provided by pyBRAvo during the reconstruction into a texte file.

In [9]:
all_stdout = '\n'.join([s.stdout for s in [pybravo_stdout_0, pybravo_stdout_1, pybravo_stdout_2]])
print(all_stdout)

Explored 16 regulators
Explored 16 regulators
Explored 25 regulators
Explored 25 regulators
Explored 28 regulators
Explored 32 regulators
Explored 34 regulators
Explored 37 regulators
Explored 40 regulators
Explored 41 regulators
Explored 41 regulators
Explored 46 regulators
Explored 51 regulators
Explored 51 regulators
Explored 51 regulators
Explored 51 regulators
Explored 52 regulators
Explored 53 regulators
Explored 71 regulators
Explored 71 regulators
Explored 73 regulators
Explored 74 regulators
Explored 74 regulators
Explored 74 regulators
Explored 74 regulators
Explored 74 regulators
Explored 74 regulators
Explored 75 regulators
Explored 75 regulators
Explored 75 regulators
Explored 77 regulators
Explored 77 regulators
Explored 77 regulators
Explored 77 regulators
Explored 77 regulators
Explored 79 regulators
Explored 79 regulators
Explored 79 regulators
Explored 79 regulators
Explored 79 regulators
Explored 79 regulators
Explored 83 regulators
Explored 83 regulators
Explored 83

In [10]:
with open('reg-demo/std.out', 'w') as outfile:
    print(all_stdout, file = outfile)

### Recovering logs

Here we read all logs saved to a file during the reconstruction for a later analysis.

In [11]:
with open('reg-demo/std.log', 'r') as logfile:
    pybravo_log_0 = logfile.read()

### Statistics extraction

Here we call several Bash and Python scripts on the results to extracts statistics on the provenance, coverage and content of the network. Some data are summed up in file `cytoscape/edges-provenance.csv` that can be loaded in Cytoscape and applied to the edges of the network.

In [12]:
%%bash
cd reg-demo

# Extract genes of the unified and non-unified results
sh scripts/get-genes-from-sif.sh out-unified.sif > out-unified.sif.genes
# Provenance of all edges in unified result
bash scripts/pybravo-provenances.sh out-unified-provenance.csv > provenance-unified.txt
# Provenance of signed edges only in unified result
bash scripts/pybravo-provenances.sh <(grep -v 'UNKNOWN' out-unified-provenance.csv) > provenance-unified-signed.txt

# Coverage
python3 scripts/compute_coverage.py out-unified.sif data/input-910.csv ../Homo_sapiens.gene_info > coverage.txt

# Provenance CSV for Cytoscape
mkdir cytoscape
echo "edge name,provenance" > cytoscape/edges-provenance.csv
sed 's/\t\(.*\)\t/ (\1) /;' out-unified-provenance.csv >> cytoscape/edges-provenance.csv

mkdir: cannot create directory 'cytoscape': File exists


In [13]:
%%bash --no-raise-error
cd reg-demo
bash scripts/stats-2.sh "out-unified.sif" "std.log"

Nodes: 1617 out-unified.sif.genes
Edges: 4209 out-unified.sif
 - signed: 3649
 - unsigned: 0
 - PART_OF: 560

Coverage: 704 genes are present over 910

Complexes: 294
Complexes involving small molecules: 108


## Using Iggy to make predictions

In this part we use the previously extracted network as a model in order to make predictions on it, using the tool [Iggy](https://github.com/bioasp/iggy).

Using some preliminary observations on some nodes, consisting in signs “$+$” (observed up-regulation) and “$-$” (observed down-regulation), Iggy tries to infer new information on the other nodes which can thus also be predicted as “$+$” or “$-$”. This call relies on Bash and Ptyhon scripts.

For this step, you need Iggy 1.4.1 installed, for instance using pip:

> ```pip install iggy==1.4.1```

Once Iggy is installed, please provide the path to it below.
Please note that this can change according to the installed sofware in your machine.
For instance, the default when only pip installed is `~/.local/bin/iggy.py`.
However, if Conda or Miniconda is installed, the default becomes `~/(path-to-conda-environment)/bin/iggy.py`.

In [14]:
#iggy_command = "${HOME}/.local/bin/iggy.py"
iggy_command = "${CONDA_PREFIX}/bin/iggy.py"

In [15]:
# Interpret Iggy command line
iggy_command = !echo "$iggy_command"
iggy_command = iggy_command[0]
print(iggy_command)

/home/max/miniconda3/envs/pybravo/bin/iggy.py


The scripts below call Iggy, with some data preparation beforehand and a comparison of the result with the original ICGC experimental data afterwards.

The printed output sums up the results of the prediction, and the consistency of these results with the initial ICGC expression data (matching or not matching predictions).

The full result can be found in file `reg-demo/cytoscape/nodes-iggy.csv` in a format that can be fed to Cytoscape in order to label nodes with their observation/prediction sign.

In [16]:
%%bash -s "$iggy_command"
export LC_ALL=C
# Make folders for execution results
cd reg-demo
mkdir iggy
cd iggy
mkdir result

# Create graph (SIF) file (removes unsigned edges)
bash ../scripts/format-iggy.sh ../out-unified.sif | grep -v 'UNKNOWN' > out-unified-iggy.sif

# Create observations (OBS) file
bash ../scripts/construct-inputs.sh out-unified-iggy.sif ../data/updownreg-v4.3-noinputs.obs > updownreg.obs

# Extract true observations (actually in network)
bash ../scripts/get-genes-from-sif.sh out-unified-iggy.sif > out-unified-iggy.sif.genes 
grep -w -f out-unified-iggy.sif.genes updownreg.obs > updownreg_true.obs

# Run Iggy workflow
bash ../scripts/workflow-iggy.sh --iggy-command $1 updownreg_true.obs out-unified-iggy.sif ../data/icgc.syn.csv 0 0 result/iggy.out > result/out-unified.tsv
cp result/out-unified.tsv ../cytoscape/nodes-iggy.csv

# Output stats
bash ../scripts/stats-result.sh --en --skip0 result/out-unified.tsv out-unified-iggy.sif > result/stats.txt
cat result/stats.txt

Total number of nodes: 1617
Total number of edges: 4209
Number of observations: 654
Number of predictions: 73
  positive (+): 37
  negative (-): 31
  no-change (0): 0
  may-up (NOT-): 0
  may-down (NOT+): 0
  change (CHANGE/NOT0): 5

Predictions found in ICGC data: 55
    +: 26
    -: 27
    0: 0
  coherent: 22
    +: 15
    -: 7
  weakly coherent: 2
    NOT-: 0
    NOT+: 0
    CHANGE: 2
  not coherent: 31
    predicted +: 11
      (should be -)
     predicted -: 20
      (should be +)
     predicted NOT-/NOT+/CHANGE: 0


mkdir: cannot create directory 'iggy': File exists
mkdir: cannot create directory 'result': File exists
